In [ ]:
import colorama
colorama.init(autoreset=True)
from email.message import EmailMessage
import cv2
import numpy as np
import colorama,smtplib
import time
import datetime
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders

net = cv2.dnn.readNet('yolov3_training_last (1).weights', 'yolov3_testing.cfg')
from colorama import Fore, Back, Style
colorama.init(autoreset=True)

classes = []
with open("classes.txt", "r") as f:
    classes = f.read().splitlines()

cap = cv2.VideoCapture(0)
font = cv2.FONT_HERSHEY_PLAIN
colors = np.random.uniform(0, 255, size=(100, 3))


def image_sender():
    sender = "cfpredictor123@gmail.com"
    password = "input()"
    reciever = "pagalavanpagal017@gmail.com"
    body = f"""
                          GARBAGE FOUND...
                          NEED TO CLEAN...
                            """
    message = MIMEMultipart()
    message.attach(MIMEText(body, 'plain'))
    email_bd = """<pre>GO TO LIVE LOCATION: 
    http://localhost:63342/PycharmProjects/map/map.html?_ijt=4d1qiecm5prrnqb14vv6r8qioa&_ij_reload=RELOAD_ON_SAVE
                       </pre>
                       """
    link = MIMEText(email_bd, 'html')
    filename = "BLOGj.png"
    attachment = open(filename, 'rb')

    attachment_package = MIMEBase('application', 'octet-stream')
    attachment_package.set_payload((attachment).read())
    encoders.encode_base64(attachment_package)
    attachment_package.add_header('Content-Disposition', "attachment; filename= " + filename)
    message.attach(attachment_package)

    text = message.as_string()

    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login(sender, password)
    print(Fore.GREEN + "\nAccount successfully verified!")
    server.sendmail(sender, reciever, text)
    server.sendmail(sender, reciever, link.as_string())
    print(Fore.YELLOW + "\nYour email has been successfully sent to", reciever)
    server.quit()

while True:
    _, img = cap.read()
    height, width, _ = img.shape

    blob = cv2.dnn.blobFromImage(img, 1/255, (416, 416), (0,0,0), swapRB=True, crop=False)
    net.setInput(blob)
    output_layers_names = net.getUnconnectedOutLayersNames()
    layerOutputs = net.forward(output_layers_names)

    boxes = []
    confidences = []
    class_ids = []

    for output in layerOutputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.2:
                center_x = int(detection[0]*width)
                center_y = int(detection[1]*height)
                w = int(detection[2]*width)
                h = int(detection[3]*height)

                x = int(center_x - w/2)
                y = int(center_y - h/2)

                boxes.append([x, y, w, h])
                confidences.append((float(confidence)))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.2, 0.4)
    k=0
    if len(indexes)>0:
        for j in range(1):

            for i in indexes.flatten():
                x, y, w, h = boxes[i]
                cv2.imwrite("BLOG" + str(k) + ".jpg", img)
                k+=1
                label = str(classes[class_ids[i]])
                confidence = str(round(confidences[i],2))
                color = colors[i]

                cv2.rectangle(img, (x,y), (x+w, y+h), color, 2)
                mini = datetime.datetime.now()
                cv2.putText(img,"HR-" + str(mini.hour) + "  MIN-" + str(mini.minute)  + "  SEC-" + str(mini.second)+"  "+ label + " " + confidence, (x, y+20), font,1,(255,255,255), 2)
                print(mini.minute)
                print(mini.hour)
                if((mini.minute==4 and (mini.second==1))or(mini.minute==2 and (mini.second==1))or(mini.minute==54 and (mini.second==1))):
                    cv2.imwrite("BLOGj.png",img)
                    image_sender()
                    j+=1

    cv2.imshow('Image', img)
    key = cv2.waitKey(1)
    if key==27:
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np
import glob
import random


# Load Yolo
net = cv2.dnn.readNet("yolov3_training_last (1).weights", "yolov3_testing.cfg")

# Name custom object
classes = ["garbage blog"]

# Images path
images_path = glob.glob(r"Garbage blob dataset/garbage_blog25.jpg")



layer_names = net.getLayerNames()
output_layers = [layer_names[i-1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

# Insert here the path of your images
random.shuffle(images_path)
# loop through all the images
for img_path in images_path:
    # Loading image
    img = cv2.imread(img_path)
    img = cv2.resize(img, None, fx=0.4, fy=0.4)
    height, width, channels = img.shape

    # Detecting objects
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

    net.setInput(blob)
    outs = net.forward(output_layers)

    # Showing informations on the screen
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.3:
                # Object detected
                print(class_id)
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    print(indexes)
    font = cv2.FONT_HERSHEY_PLAIN
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            color = colors[class_ids[i]]
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv2.putText(img, label, (x, y + 30), font, 3, color, 2)


    cv2.imshow("Image", img)
    key = cv2.waitKey(0)

cv2.destroyAllWindows()